# Plot3D Read/Write Formats Tutorial

This tutorial demonstrates the different file format options available for reading and writing Plot3D files:

1. **Binary vs ASCII** - Choose between compact binary or human-readable ASCII format
2. **Endianness** - Support for both little-endian and big-endian byte ordering
3. **Fortran Format** - Fortran unformatted binary with record markers
4. **Precision** - Single (32-bit) or double (64-bit) precision floating point

## Install Dependencies

Run this cell if you haven't installed plot3d yet:

In [ ]:
# !pip install plot3d

## Import Libraries

In [ ]:
import numpy as np
from plot3d import Block, read_plot3D, write_plot3D

## Create a Sample Block

Let's create a simple 3D block for testing. This creates a unit cube mesh.

In [ ]:
# Create a simple 10x10x10 unit cube mesh
ni, nj, nk = 10, 10, 10

# Create coordinate arrays
x = np.linspace(0, 1, ni)
y = np.linspace(0, 1, nj)
z = np.linspace(0, 1, nk)

# Create 3D mesh grids
X, Y, Z = np.meshgrid(x, y, z, indexing='ij')

# Create a Block object
block = Block(X=X, Y=Y, Z=Z)

print(f"Block dimensions: IMAX={block.IMAX}, JMAX={block.JMAX}, KMAX={block.KMAX}")
print(f"Total points: {block.IMAX * block.JMAX * block.KMAX}")

## 1. Binary Format (Default)

The default format is binary with little-endian byte order and double precision. This is the most compact format.

In [ ]:
# Write binary (default)
write_plot3D('mesh_binary.xyz', [block])

# Read it back
blocks_read = read_plot3D('mesh_binary.xyz')

# Verify the data matches
print(f"Original X range: [{block.X.min():.6f}, {block.X.max():.6f}]")
print(f"Read X range:     [{blocks_read[0].X.min():.6f}, {blocks_read[0].X.max():.6f}]")
print(f"Arrays match: {np.allclose(block.X, blocks_read[0].X)}")

## 2. ASCII Format

ASCII format is human-readable and uses scientific notation for full precision. Set `binary=False` to use ASCII format.

In [ ]:
# Write ASCII format
write_plot3D('mesh_ascii.xyz', [block], binary=False)

# Read it back
blocks_ascii = read_plot3D('mesh_ascii.xyz', binary=False)

# Verify
print(f"Arrays match: {np.allclose(block.X, blocks_ascii[0].X)}")

# Let's look at the first few lines of the ASCII file
print("\nFirst 10 lines of ASCII file:")
with open('mesh_ascii.xyz', 'r') as f:
    for i, line in enumerate(f):
        if i < 10:
            print(line.rstrip())
        else:
            break

## 3. Single Precision

For smaller file sizes (at the cost of precision), use single precision (32-bit floats instead of 64-bit doubles).

In [ ]:
# Write single precision binary
write_plot3D('mesh_single.xyz', [block], double_precision=False)

# Read it back (must specify read_double=False)
blocks_single = read_plot3D('mesh_single.xyz', read_double=False)

# Verify (note: may have small differences due to precision)
print(f"Arrays match (within single precision): {np.allclose(block.X, blocks_single[0].X, rtol=1e-6)}")

# Compare file sizes
import os
size_double = os.path.getsize('mesh_binary.xyz')
size_single = os.path.getsize('mesh_single.xyz')
print(f"\nDouble precision file size: {size_double} bytes")
print(f"Single precision file size: {size_single} bytes")
print(f"Size reduction: {(1 - size_single/size_double)*100:.1f}%")

## 4. Big-Endian Byte Order

Some legacy systems or codes may require big-endian byte ordering. Use `big_endian=True` for compatibility.

In [ ]:
# Write big-endian binary
write_plot3D('mesh_big_endian.xyz', [block], big_endian=True)

# Read it back (must specify big_endian=True)
blocks_be = read_plot3D('mesh_big_endian.xyz', big_endian=True)

# Verify
print(f"Arrays match: {np.allclose(block.X, blocks_be[0].X)}")

# What happens if we read with wrong endianness?
blocks_wrong = read_plot3D('mesh_big_endian.xyz', big_endian=False)
print(f"\nReading big-endian file with little-endian setting:")
print(f"X range (wrong): [{blocks_wrong[0].X.min():.6e}, {blocks_wrong[0].X.max():.6e}]")
print("Note: Values are garbage because byte order is interpreted incorrectly!")

## 5. Fortran Unformatted Binary

Fortran programs often use "unformatted" binary files which include record length markers. This format is commonly used by legacy CFD codes.

Each record is wrapped with 4-byte length markers:
```
[4-byte length][data][4-byte length]
```

Use `fortran=True` to read/write this format.

In [ ]:
# Write Fortran unformatted binary
write_plot3D('mesh_fortran.xyz', [block], fortran=True)

# Read it back
blocks_fortran = read_plot3D('mesh_fortran.xyz', fortran=True)

# Verify
print(f"Arrays match: {np.allclose(block.X, blocks_fortran[0].X)}")

# Compare file sizes (Fortran files are slightly larger due to record markers)
size_fortran = os.path.getsize('mesh_fortran.xyz')
print(f"\nC binary file size:       {size_double} bytes")
print(f"Fortran binary file size: {size_fortran} bytes")
print(f"Overhead from record markers: {size_fortran - size_double} bytes")

## 6. Fortran Format with Single Precision

In [ ]:
# Write Fortran format with single precision
write_plot3D('mesh_fortran_single.xyz', [block], fortran=True, double_precision=False)

# Read it back
blocks_fs = read_plot3D('mesh_fortran_single.xyz', fortran=True, read_double=False)

# Verify
print(f"Arrays match (within single precision): {np.allclose(block.X, blocks_fs[0].X, rtol=1e-6)}")

## 7. Multiple Blocks

All format options work with multi-block meshes.

In [ ]:
# Create a second block (shifted in X)
X2 = X + 1.0  # Shift by 1 in X direction
block2 = Block(X=X2, Y=Y, Z=Z)

# Write multiple blocks
write_plot3D('mesh_multiblock.xyz', [block, block2])

# Read back
blocks_multi = read_plot3D('mesh_multiblock.xyz')

print(f"Number of blocks written: 2")
print(f"Number of blocks read: {len(blocks_multi)}")
print(f"Block 1 X range: [{blocks_multi[0].X.min():.2f}, {blocks_multi[0].X.max():.2f}]")
print(f"Block 2 X range: [{blocks_multi[1].X.min():.2f}, {blocks_multi[1].X.max():.2f}]")

## Summary: Function Signatures

### write_plot3D
```python
write_plot3D(
    filename: str,           # Output filename
    blocks: List[Block],     # List of Block objects to write
    binary: bool = True,     # True for binary, False for ASCII
    big_endian: bool = False,# True for big-endian byte order
    double_precision: bool = True,  # True for 64-bit, False for 32-bit
    fortran: bool = False,   # True for Fortran unformatted binary
    batch_size: int = 100    # Buffer size for writing
)
```

### read_plot3D
```python
read_plot3D(
    filename: str,           # Input filename
    binary: bool = True,     # True for binary, False for ASCII
    big_endian: bool = False,# True for big-endian byte order
    read_double: bool = True,# True for 64-bit, False for 32-bit
    fortran: bool = False    # True for Fortran unformatted binary
) -> List[Block]
```

## Cleanup

Remove the test files created in this tutorial.

In [ ]:
import os

test_files = [
    'mesh_binary.xyz',
    'mesh_ascii.xyz', 
    'mesh_single.xyz',
    'mesh_big_endian.xyz',
    'mesh_fortran.xyz',
    'mesh_fortran_single.xyz',
    'mesh_multiblock.xyz'
]

for f in test_files:
    if os.path.exists(f):
        os.remove(f)
        print(f"Removed {f}")